In [1]:
import os
from pathlib import Path
import numpy as np
import scipy
Downloads2 = str(os.path.join(Path.home(), 'downloads','reactions.txt'))
reactions = np.genfromtxt(fname=Downloads2, delimiter='/n', dtype='unicode')
print(reactions)

#--IMPORTED DATA FROM FILE--#

def stoichiometric_coefficient(molecule):
    position = 0
    coefficient = ''
    while position < len(molecule):
        if molecule[position].isdigit():
            coefficient += molecule[position]
        else:
            break
        position += 1
    if coefficient == '':
        just_molecule = molecule
        return (1,just_molecule)
    else:
        just_molecule = molecule[len(coefficient):]
        return (int(coefficient),just_molecule)
def count_atoms(molecule):
    charge = 0
    poscharge = molecule.find('+')
    negcharge = molecule.find('-')
    if poscharge != -1:
        charge = molecule.split('+')[1]
        if charge == '':
            charge = 1
        else:
            charge = int(charge)
        molecule = molecule.split('+')[0]
    if negcharge != -1:
        charge = molecule.split('-')[1]
        if charge == '':
            charge = -1
        else:
            charge = -int(charge)
        molecule = molecule.split('-')[0]
    position = 0
    atoms = {}
    if charge != 0:
        atoms['e'] = charge
    current_atom = ''
    current_number = ''
    for x in molecule:
        if x.isupper():
            if current_atom != '':
                if current_number == '': 
                    if current_atom in atoms.keys():
                        atoms[current_atom] += 1
                    else:
                        atoms[current_atom] = 1
                else:
                    if current_atom in atoms.keys():
                        atoms[current_atom] += int(current_number)
                    else:
                        atoms[current_atom] = int(current_number)
                current_atom = ''
                current_number = ''
            current_atom += x
        elif x.islower():
            current_atom += x
        else:
            current_number += x
    if current_atom != '':
        if current_number == '':
            if current_atom in atoms.keys():
                atoms[current_atom] += 1
            else:
                atoms[current_atom] = 1
        else:
            if current_atom in atoms.keys():
                atoms[current_atom] += int(current_number)
            else:
                atoms[current_atom] = int(current_number)
    return atoms

#--DEFINED IMPORTANT VARIABLES --#
#--BEGIN MAJOR FOR LOOP--#

for x in reactions:
    all_reactants = []
    all_products = []
    reactants_together = x.split(" -> ")[0]
    products_together = x.split(" -> ")[1]
    reactants = reactants_together.split(" + ")
    products = products_together.split(" + ")

    #--SPLITS EACH REACTION INPUT--#
    
    allreactants = {}
    allproducts = {}
    for reactant in reactants:
        (n,mol) = stoichiometric_coefficient(reactant)
        atoms = count_atoms(mol)
        for atom in atoms.keys():
            if atom in allreactants.keys():
                allreactants[atom] += n * atoms[atom]
            else:
                allreactants[atom] = n * atoms[atom]
    for product in products:
        (n,mol) = stoichiometric_coefficient(product)
        atoms = count_atoms(mol)
        for atom in atoms.keys():
            if atom in allproducts.keys():
                allproducts[atom] += n * atoms[atom]
            else:
                allproducts[atom] = n * atoms[atom]
    atoms = sorted(allreactants.keys())
    nrows = len(atoms)
    print ('Number of rows:',nrows)
    ncolumns = len(reactants) + len(products) - 1
    print ('Number of columns:',ncolumns)
    mat = np.zeros((nrows,ncolumns))
    column = 0
    for compound in reactants[1:]:
        (n,mol) = stoichiometric_coefficient(compound)
        atoms_here = count_atoms(mol)
        for atomtype in range(len(atoms)):
            if atoms[atomtype] in atoms_here.keys():
                mat[atomtype,column] = -atoms_here[atoms[atomtype]]
                #print(mol,atomtype,atoms[atomtype],atoms_here[atoms[atomtype]])
        column += 1
    for compound in products:
        (n,mol) = stoichiometric_coefficient(compound)
        atoms_here = count_atoms(mol)
        for atomtype in range(len(atoms)):
            if atoms[atomtype] in atoms_here.keys():
                mat[atomtype,column] = atoms_here[atoms[atomtype]]
                #print(mol,atomtype,atoms[atomtype],atoms_here[atoms[atomtype]])
        column += 1
    print(mat)
    rhs = np.zeros(nrows)
    (n,mol) = stoichiometric_coefficient(reactants[0])
    atoms_here = count_atoms(mol)
    for atomtype in range(len(atoms)):
        if atoms[atomtype] in atoms_here.keys():
            rhs[atomtype] = atoms_here[atoms[atomtype]]
    #print(rhs)
    res = scipy.linalg.lstsq(mat, rhs)
    print(res)


['C2H5OH + O2 -> CO2 + H2O' 'CrO4-2 + H+ -> Cr2O7-2 + H2O'
 'KMnO4 + KI + HCl -> KCl + MnCl2 + I2 + H2O'
 'MnO4- + I- + H+ -> Mn+2 + I2 + H2O'
 'HgS2C2N2 + O2 -> HgS + CO2 + SO2 + C3N4']
Number of rows: 3
Number of columns: 3
[[ 0.  1.  0.]
 [ 0.  0.  2.]
 [-2.  2.  1.]]
(array([3., 2., 3.]), array([], dtype=float64), 3, array([3.17740968, 1.85577251, 0.67836283]))
Number of rows: 4
Number of columns: 3
[[ 0.  2.  0.]
 [-1.  0.  2.]
 [ 0.  7.  1.]
 [-1. -2.  0.]]
(array([1. , 0.5, 0.5]), 5.514772727388619e-31, 3, array([7.61438058, 2.34387987, 0.7262476 ]))
Number of rows: 6
Number of columns: 6
[[ 0. -1.  1.  2.  0.  0.]
 [ 0. -1.  0.  0.  0.  2.]
 [-1.  0.  0.  0.  2.  0.]
 [-1.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.]]
(array([5. , 8. , 6. , 1. , 2.5, 4. ]), array([], dtype=float64), 6, array([2.71894754, 2.33521131, 2.27758922, 1.26979359, 0.56292856,
       0.19348227]))
Number of rows: 5
Number of columns: 5
[[ 0. -1.  0.  0.  2.]
 [-1.  0.  0.  